In [1]:
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Embedding, Input, Reshape, Dense, Lambda, Dropout, Flatten, MaxPooling2D,MaxPooling1D, Conv2D, \
    BatchNormalization, LSTM, GRU, Embedding, Conv1D,SimpleRNN, TimeDistributed, SimpleRNN, ConvLSTM2D, CuDNNLSTM, \
    CuDNNGRU, Bidirectional, Activation, MaxPool2D, AveragePooling2D, RepeatVector, AtrousConvolution1D, MaxPooling1D, Activation
from mimo_channels_data_generator2 import RandomChannelMimoDataGenerator
from keras.callbacks import TensorBoard
#from matplotlib import pyplot as plt
import argparse
from keras.models import model_from_json
from keras.constraints import max_norm
import numpy.linalg as la
from keras import backend as K
import os
import shutil
import sys
import json
from datetime import datetime
from scipy.io import savemat, loadmat

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 1
np.random.seed(seed)

# training parameters
epochs = 10

# Parameters
global Nt
global Nr
Nr = 8  # num of Rx antennas, will be larger than Nt for uplink massive MIMO
Nt = 8  # num of Tx antennas
# the sample is a measurement of Y values, and their collection composes an example. The channel estimation
min_randomized_snr_db = -1
max_randomized_snr_db = 1

# must be done per example, each one having a matrix of Nr x numSamplesPerExample of complex numbers
numSamplesPerExample = 256 # number of channel uses, input and output pairs
numExamplesWithFixedChannel = 1 # if wants a gradient calculated many times with same channel
numSamplesPerFixedChannel = numExamplesWithFixedChannel * numSamplesPerExample  # coherence time
batch_size = 5 # obs: it may make sense to have the batch size equals the coherence time
##### consider to use train_test_split from sklearn
num_test_examples = 500  # for evaluating in the end, after training
num_validation_examples =  200 # get small number to avoid slowing down the simulation, test in the end
num_training_examples = 1000

method = 'manual'
file = 'channel_data60Ghz_mob.mat'

In [10]:
training_generator = RandomChannelMimoDataGenerator(batch_size=batch_size, Nr=Nr, Nt=Nt,
                                                    #num_clusters=num_clusters,
                                                    numSamplesPerFixedChannel=numSamplesPerFixedChannel,
                                                    #numSamplesPerExample=numSamplesPerExample, SNRdB=SNRdB,
                                                    numSamplesPerExample=numSamplesPerExample,
                                                    # method='random')
                                                    method=method,
                                                    file = file)
training_generator.randomize_SNR = True
training_generator.min_randomized_snr_db = min_randomized_snr_db
training_generator.max_randomized_snr_db = max_randomized_snr_db

input_train, output_train = training_generator.get_examples(num_training_examples)
#savemat(f"C:\\Users\\wesin\\Documents\\Wesin\\Doutorado\\channel-estimation\\datasets\\1-bit\\mimo8x8_60Hz_mob.mat", {'inputs': input_train, 'outputs': output_train})
#input_train = input_train.reshape((-1,1,2*Nr))
print(input_train.shape)
print(output_train.shape)
input_val, output_val = training_generator.get_examples(num_validation_examples)
#input_val = input_val.reshape((-1,1,2*Nr))
print(input_val.shape)

global H_normalization_factor
H_normalization_factor = np.sqrt(Nr * Nt)

(5000, 256, 16)
(5000, 16, 8)
(1000, 256, 16)


In [11]:
# real / compl as twice number of rows
input_shape = (numSamplesPerExample, 2 * (Nr))
output_dim = (2 * Nr, Nt)

numInputs = np.prod(input_shape)
numOutputs = np.prod(output_dim)
print(numInputs, " ", numOutputs)

4096   128


In [12]:
input_train = input_train.reshape((-1,256*16))
output_train = output_train.reshape((-1,16*8))
input_val = input_val.reshape((-1,256*16))
output_val = output_val.reshape((-1,16*8))

In [13]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(input_train, output_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
from sklearn.metrics import mean_squared_error
channel_predictions = model.predict(input_val)
lin_mse = mean_squared_error(output_val, channel_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

0.4132191165795876


In [15]:
# test with disjoint test set
#SNRdB_values = np.arange(-30,20)
SNRdB_values = np.arange(-21,22,3)
training_generator.randomize_SNR = False
training_generator.method = 'manual'
all_nmse_db_average = np.zeros((SNRdB_values.shape))
all_nmse_db_min = np.zeros((SNRdB_values.shape))
all_nmse_db_max = np.zeros((SNRdB_values.shape))

In [10]:
#evaluate for several SNR values
it=0
for SNRdB in SNRdB_values:
    training_generator.SNRdB = SNRdB
    #get rid of the last example in the training_generator's memory (flush it)
    testInput, testOutput = training_generator.get_examples(1)
    #now get the actual examples:
    testInput, testOutput = training_generator.get_examples(num_test_examples)
    savemat(f"C:\\Users\\wesin\\Documents\\Wesin\\Doutorado\\channel-estimation\\datasets\\1-bit\\testing2\\mimo8x8_{SNRdB}snr_60Hz_mob.mat", {'inputs': testInput, 'outputs': testOutput})
    
    testInput = testInput.reshape((-1,256*16))
    testOutput = testOutput.reshape((-1,16*8))
    
    predictedOutput = model.predict(testInput)
    
    error = testOutput - predictedOutput

    mseTest = np.mean(error[:] ** 2)
    print('overall MSE = ', mseTest)
    mean_nmse = mseTest / (Nr * Nt)
    print('overall NMSE = ', mean_nmse)
    nmses = np.zeros((num_test_examples,))
    for i in range(num_test_examples):
        this_H = testInput[i]
        this_error = error[i]
        nmses[i] = np.mean(this_error[:] ** 2) / np.mean(this_H[:] ** 2)

    print('NMSE: mean', np.mean(nmses), 'min', np.min(nmses), 'max', np.max(nmses))
    nmses_db = 10 * np.log10(nmses)
    print('NMSE dB: mean', np.mean(nmses_db), 'min', np.min(nmses_db), 'max', np.max(nmses_db))
    all_nmse_db_average[it] = np.mean(nmses_db)
    all_nmse_db_min[it] = np.min(nmses_db)
    all_nmse_db_max[it] = np.max(nmses_db)

overall MSE =  0.624452281316076
overall NMSE =  0.009757066895563687
NMSE: mean 0.6258731571950327 min 0.43297801778353506 max 0.8251925773005846
NMSE dB: mean -2.0663320136346863 min -3.6353415214005294 max -0.8344468720869624
overall MSE =  0.5826437540204535
overall NMSE =  0.009103808656569587
NMSE: mean 0.5851882109806723 min 0.38732597460184887 max 0.7903120845381982
NMSE dB: mean -2.3538602758426195 min -4.119233776640667 max -1.0220137728115926
overall MSE =  0.5246188487887836
overall NMSE =  0.008197169512324744
NMSE: mean 0.5234630719061449 min 0.388864644097046 max 0.7231467904913476
NMSE dB: mean -2.8421180177971017 min -4.102015414798828 max -1.4077353696596582
overall MSE =  0.4562227324161927
overall NMSE =  0.007128480194003011
NMSE: mean 0.45142974738500274 min 0.2680948045190651 max 0.7091336264550112
NMSE dB: mean -3.4876342778328198 min -5.717116022445135 max -1.4927192029800125
overall MSE =  0.3736659053003656
overall NMSE =  0.005838529770318212
NMSE: mean 0.37

# Using offline data

In [3]:
from scipy.io import loadmat
mat = loadmat("C:\\Users\\wesin\\Documents\\Wesin\\Doutorado\\channel-estimation\\datasets\\1-bit\\mimo8x8_60Hz_mob")
inputs, outputs = [mat[key] for key in ["inputs", "outputs"]]

In [4]:
input_train = inputs.reshape((-1,256*16))
output_train = outputs.reshape((-1,16*8))
print(input_train.shape)
print(output_train.shape)

(5000, 4096)
(5000, 128)


In [17]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=5)
model.fit(input_train, output_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [18]:
from sklearn.metrics import mean_squared_error
channel_predictions = model.predict(input_train[0:1000])
lin_mse = mean_squared_error(output_train[0:1000], channel_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

0.2609538654778709


In [15]:
# test with disjoint test set
#SNRdB_values = np.arange(-30,20)
SNRdB_values = np.arange(-21,22,3)
all_nmse_db_average = np.zeros((SNRdB_values.shape))
all_nmse_db_min = np.zeros((SNRdB_values.shape))
all_nmse_db_max = np.zeros((SNRdB_values.shape))

In [19]:
#evaluate for several SNR values
Nr = 8  # num of Rx antennas, will be larger than Nt for uplink massive MIMO
Nt = 8  # num of Tx antennas
num_test_examples = 2500
it=0
for SNRdB in SNRdB_values:
    mat = loadmat(f"C:\\Users\\wesin\\Documents\\Wesin\\Doutorado\\channel-estimation\\datasets\\1-bit\\testing2\\mimo8x8_{SNRdB}snr_60Hz_mob.mat")
    inputs, outputs = [mat[key] for key in ["inputs", "outputs"]]
    
    testInput = inputs.reshape((-1,256*16))
    testOutput = outputs.reshape((-1,16*8))
    
    predictedOutput = model.predict(testInput)
    
    error = testOutput - predictedOutput

    mseTest = np.mean(error[:] ** 2)
    print('overall MSE = ', mseTest)
    mean_nmse = mseTest / (Nr * Nt)
    print('overall NMSE = ', mean_nmse)
    nmses = np.zeros((num_test_examples,))
    for i in range(num_test_examples):
        this_H = testInput[i]
        this_error = error[i]
        nmses[i] = np.mean(this_error[:] ** 2) / np.mean(this_H[:] ** 2)

    print('NMSE: mean', np.mean(nmses), 'min', np.min(nmses), 'max', np.max(nmses))
    nmses_db = 10 * np.log10(nmses)
    print('NMSE dB: mean', np.mean(nmses_db), 'min', np.min(nmses_db), 'max', np.max(nmses_db))
    all_nmse_db_average[it] = np.mean(nmses_db)
    all_nmse_db_min[it] = np.min(nmses_db)
    all_nmse_db_max[it] = np.max(nmses_db)

overall MSE =  0.46330205956495313
overall NMSE =  0.007239094680702393
NMSE: mean 0.4633020595649532 min 0.026233850673760148 max 0.789020841012996
NMSE dB: mean -3.704212518179597 min -15.8113795784177 max -1.0291152528563285
overall MSE =  0.3788423661189883
overall NMSE =  0.005919411970609192
NMSE: mean 0.3788423661189883 min 0.012224496844278718 max 0.7969019528268194
NMSE dB: mean -4.973044960162028 min -19.127690072350404 max -0.9859510892468841
overall MSE =  0.2951589195259525
overall NMSE =  0.004611858117593008
NMSE: mean 0.29515891952595263 min 0.006820195568827765 max 0.7988458142122755
NMSE dB: mean -6.5974871272023305 min -21.662031717885398 max -0.9753703607846874
overall MSE =  0.22082171434869352
overall NMSE =  0.0034503392866983363
NMSE: mean 0.2208217143486936 min 0.0038734179310085377 max 0.7788986649710815
NMSE dB: mean -8.215279603723273 min -24.11905641320521 max -1.0851904053869204
overall MSE =  0.17736486735922782
overall NMSE =  0.0027713260524879347
NMSE: